### Laboratorio #5 – Threat hunting


#### Jose Andres Hernandez Guerra 20053

1. Cargue las librerías y archivos a utilizar en la misma ubicación

Cargue la información del archivo large_even.json en una lista, muestre la cantidad de
registros total (deben ser 746, 909). Este es nuestro tráfico inicial!

In [1]:
import json
import tldextract


# Cargamos el archivo de datos ubicado en "./Archivos/large_eve.json" y lo guardamos en una lista

data = []
with open("./Archivos/large_eve.json", 'r') as file:

    for line in file:
    
        record = json.loads(line) 
    
        data.append(record)

# Mostramos datos importantes sobre el archivo

print("Cantidad de eventos: ", len(data))

# Mostramos los primeros 5 eventos

for i in range(5):

    print(data[i])

Cantidad de eventos:  746909
{'timestamp': '2017-07-22T17:33:15.784100-0500', 'flow_id': 283570730437312, 'pcap_cnt': 18519, 'event_type': 'alert', 'vlan': 130, 'src_ip': '192.168.203.200', 'src_port': 2328, 'dest_ip': '192.181.145.109', 'dest_port': 445, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2001569, 'rev': 15, 'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection', 'category': 'Misc activity', 'severity': 3}}
{'timestamp': '2017-07-22T17:33:16.571273-0500', 'flow_id': 1519048138171115, 'pcap_cnt': 21531, 'event_type': 'ssh', 'vlan': 140, 'src_ip': '192.168.204.60', 'src_port': 62990, 'dest_ip': '192.168.26.254', 'dest_port': 22, 'proto': 'TCP', 'ssh': {'client': {'proto_version': '2.0', 'software_version': 'OpenSSH_5.0'}, 'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}
{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 1

Debido a que estamos buscando dominios web, del total de registros, solamente estamos
interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS.

In [2]:
eventos_especificos_dns = []

# Buscamos eventos que tengan el campo "dns" en el campo "event_type"

for evento in data:

    if evento["event_type"] == "dns":

        eventos_especificos_dns.append(evento)

# Mostramos la cantidad de eventos que cumplen con la condición

print("Cantidad de eventos con el campo 'dns' en el campo 'event_type': ", len(eventos_especificos_dns))

# Mostramos los primeros 5 eventos que cumplen con la condición

for i in range(5):

    print(eventos_especificos_dns[i])

Cantidad de eventos con el campo 'dns' en el campo 'event_type':  15749
{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:27.379891-0500', 'flow_id': 578544790391795, 'pcap_cnt': 54519, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.

Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad
mucho más manejable, pero aún se debe seguir depurando la información a buscar.

In [3]:
import pandas as pd

In [4]:
# Normalizamos los datos

df_normalized = pd.json_normalize(data)

print(df_normalized.head())

# Mostramos los nombres de las columnas

print(df_normalized.columns)

                         timestamp       flow_id  pcap_cnt event_type   vlan  \
0  2017-07-22T17:33:15.784100-0500  2.835707e+14   18519.0      alert  130.0   
1  2017-07-22T17:33:16.571273-0500  1.519048e+15   21531.0        ssh  140.0   
2  2017-07-22T17:33:16.661646-0500  1.327836e+15   22269.0        dns  110.0   
3  2017-07-22T17:33:17.561063-0500  9.632021e+14   28731.0       http  140.0   
4  2017-07-22T17:33:17.595416-0500  9.632021e+14   28994.0   fileinfo  140.0   

                                    src_ip  src_port  \
0                          192.168.203.200    2328.0   
1                           192.168.204.60   62990.0   
2  2001:0dbb:0c18:0011:0260:6eff:fe30:0863   59680.0   
3                           192.168.204.59   38011.0   
4                           192.168.24.201      80.0   

                                   dest_ip  dest_port proto  ...  \
0                          192.181.145.109      445.0   TCP  ...   
1                           192.168.26.254    

In [5]:

registros_dns_A = df_normalized[df_normalized['dns.rrtype'] == 'A']

cantidad_registros_dns_A = len(registros_dns_A)

print(f"En este dataset hay {cantidad_registros_dns_A} registros DNS tipo A")

dominios_unicos = registros_dns_A['dns.rrname'].unique()

cantidad_dominios_unicos = len(dominios_unicos)

print(f"En este dataset hay {cantidad_dominios_unicos} dominios únicos")

En este dataset hay 2849 registros DNS tipo A
En este dataset hay 177 dominios únicos


In [6]:
def obtener_tld(dominio):

    resultado = tldextract.extract(dominio)

    tld = resultado.domain
    
    if resultado.suffix:

        tld += '.' + resultado.suffix

    return tld

In [8]:
# Agregamos una columna con el TLD de cada dominio

registros_dns_A['tld'] = registros_dns_A['dns.rrname'].apply(obtener_tld)

print(registros_dns_A.head())

# Mostramos los TLDs más comunes

tlds_comunes = registros_dns_A['tld'].value_counts()

print(tlds_comunes.head())

# Mostramos los TLDs menos comunes

print(tlds_comunes.tail())

# Utilizamos la funcion con unos cuantos ejemplos

print(obtener_tld("www.google.com"))

print(obtener_tld("www.google.com.ar"))

print(obtener_tld("www.google.com.ar.ar"))

print(obtener_tld("www.google"))

print(obtener_tld("www.google."))

print(obtener_tld("www.google.com."))

                          timestamp       flow_id  pcap_cnt event_type   vlan  \
2   2017-07-22T17:33:16.661646-0500  1.327836e+15   22269.0        dns  110.0   
38  2017-07-22T17:33:24.990320-0500  2.022925e+15   54352.0        dns  110.0   
39  2017-07-22T17:33:27.379891-0500  5.785448e+14   54519.0        dns  150.0   
43  2017-07-22T17:33:36.672785-0500  2.379195e+14   55496.0        dns  110.0   
44  2017-07-22T17:33:38.537426-0500  2.167545e+15   55687.0        dns  180.0   

                                     src_ip  src_port  \
2   2001:0dbb:0c18:0011:0260:6eff:fe30:0863   59680.0   
38  2001:0dbb:0c18:0011:0260:6eff:fe30:0863   38051.0   
39                          192.168.205.170   31393.0   
43  2001:0dbb:0c18:0011:0260:6eff:fe30:0863   41663.0   
44                           192.168.198.62   35092.0   

                                    dest_ip  dest_port proto  ...  \
2   2001:0500:0001:0000:0000:0000:803f:0235       53.0   UDP  ...   
38  2001:0500:0003:0000:0000:000

/var/folders/hr/ygkbzwn15v3czjk9nvrdsjyc0000gn/T/ipykernel_6941/2775260445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  registros_dns_A['tld'] = registros_dns_A['dns.rrname'].apply(obtener_tld)


In [9]:
tlds = [obtener_tld(dominio) for dominio in dominios_unicos]

tlds_df = pd.DataFrame(tlds, columns=['domain_tld'])
tlds_df.head(10)

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com
5,metasploit.com
6,aol.com
7,aoltw.net
8,aol.com
9,aol.com


In [10]:
from Archivos.clasificador import clasificacion

df = clasificacion(tlds_df)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'd_common_en_words.pickle'